In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import requests
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import json
from copy import deepcopy


from matplotlib import rc
import matplotlib.ticker as tick

from rnn import *
from datasets import *
from torch.utils.data import DataLoader, random_split
from sklearn import metrics

set_random_seed(7)

In [2]:
oids, labels = get_only_r_oids('akb.ztf.snad.space.json')
data = EmbsSequenceData(oids, labels, label_type='float', path='embeddings_100ep/')

fold1, fold2, fold3, fold4, fold5 = random_split(data, [0.2, 0.2, 0.2, 0.2, 0.2])
folds = [fold1, fold2, fold3, fold4, fold5]

In [3]:
#output, labels = [], []
def get_result_kfold(model, model_name):
    result = {'roc_auc':[], 'tpr':[], 'fpr':[], 'f1':[],
              'thresholds':[], 'precision':[], 'recall':[],
              'accuracy':[]}
    bucket_boundaries = [200, 400, 600, 800]
    for k, fold in enumerate(folds):
    #concat_folds = ConcatDataset(folds[:k] + folds[k+1:])
    #train_sampler = BySequenceLengthSampler(concat_folds, bucket_boundaries, 32, drop_last=False, shuffle=True)
        test_sampler = BySequenceLengthSampler(fold, bucket_boundaries, 32, drop_last=False, shuffle=False)


    #train_loader = DataLoader(concat_folds, batch_size=1, 
    #                    batch_sampler=train_sampler, 
    #                    num_workers=16,
    #                    collate_fn=collate,
    #                    drop_last=False, pin_memory=False)

        test_loader = DataLoader(fold, batch_size=1, 
                        batch_sampler=test_sampler, 
                        num_workers=4,
                        collate_fn=collate,
                        drop_last=False, pin_memory=False)
        
        model.load_state_dict(torch.load(f'trained_models/rnn/kfold/{model_name}/model{k}.zip'))
        model.eval()
        if model.out_size == 2:
            out, gt, _ = get_pred(model, test_loader)
        else:
            out, gt = get_pred(model, test_loader)
        fpr, tpr, thresholds = metrics.roc_curve(gt, out)
        result['tpr'].append(tpr)
        result['fpr'].append(fpr)
        result['thresholds'].append(thresholds)
        result['roc_auc'].append(metrics.roc_auc_score(gt, out))
        f1scores = []
        for tr in thresholds:
            current_predict = (out >=tr).long()
            f1scores.append(metrics.f1_score(gt, current_predict))
        #result['f1'].append(f1scores)
        result['f1'].append(np.max(f1scores))
        ind = np.argmax(f1scores)
        cur_best_thr = result['thresholds'][-1][ind] #best thresholds for i fold
        result['precision'].append(metrics.precision_score(gt, (out >= cur_best_thr).long()))
        result['recall'].append(metrics.recall_score(gt, (out >= cur_best_thr).long()))
        result['accuracy'].append(metrics.accuracy_score(gt, (out >= cur_best_thr).long()))
    return result

In [4]:
model = RBclassifier(hidden_size=128, latent_dim=36, rnn_type='GRU', out_size=2, bidirectional=True)
results = get_result_kfold(model, 'wd_2dir_nomish')

In [5]:
print('-------------ROC AUC-------------')
print(f"{np.mean(results['roc_auc']):.3f} +- {np.std(results['roc_auc']):.3f}")
    

print('-------------Accuracy-------------')
print(f"{np.mean(results['accuracy']):.3f} +- {np.std(results['accuracy']):.3f}")

    
print('-------------F1-score-------------')
print(f"{np.mean(results['f1']):.3f} +- {np.std(results['f1']):.3f}")

-------------ROC AUC-------------
0.856 +- 0.010
-------------Accuracy-------------
0.802 +- 0.023
-------------F1-score-------------
0.804 +- 0.011
